## A toy example of using the dgrid_components.py module in risktables, for both plots inside a jupyter notebook, and to run a web app from a jupyter notebook

In [1]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import datetime
import pandas as pd
import numpy as np
import importlib

from dashgrid import dgrid_components as dgc
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import pathlib
SYSTEM_HOME = pathlib.Path.home()

### Generate a Pandas DataFrame ( ```df_pseudo```), with open, high, low and close data generated from a random walk process.

In [2]:
all_days=1000
end_date = datetime.datetime.now()
beg_date = end_date - datetime.timedelta(all_days)
date_series = pd.bdate_range(beg_date,end_date)
trade_dates = date_series.astype(str).str.replace('-','').astype(int)
n = len(trade_dates)
changes = np.random.lognormal(0,.15/256**.5,n-1)
initial = np.array([100.0])
closes = np.cumprod(np.append(initial,changes)).round(2)
open_ranges = np.random.lognormal(0,.3/256**.5,n)
opens = (closes * open_ranges).round(2)
low_ranges = np.random.lognormal(.1,.2/256**.5,n)
lows = np.array([min(x,y) for x,y in zip(opens,closes)]) - low_ranges
lows = lows.round(2)
high_ranges = np.random.lognormal(.1,.2/256**.5,n)
highs = np.array([max(x,y) for x,y in zip(opens,closes)]) + high_ranges
highs = highs.round(2)

volume_changes = np.random.lognormal(0,.50/256**.5,n-1)
initial_volume = np.array([1000000.0]) * np.random.lognormal(0,.15/256**.5)
volumes = np.cumprod(np.append(initial_volume,volume_changes)).round(0)


df_pseudo = dgc.make_df({'date':trade_dates,'open':opens,'high':highs,'low':lows,
                  'close':closes,'volume':volumes})


### Plot candlesticks using the data from ```df_pseudo```

In [3]:
importlib.reload(dgc)

<module 'dashgrid.dgrid_components' from '/Users/bperlman1/Documents/billybyte/pyliverisk/dashgrid/dashgrid/dgrid_components.py'>

In [4]:
fig1 = dgc.PlotlyCandles(df_pseudo.iloc[-50:],number_of_ticks_display=20,title='Random Walk')
iplot(fig1.get_figure())


In [5]:
import json
import plotly
f = json.dumps(fig1.get_figure(), cls=plotly.utils.PlotlyJSONEncoder)
open(f'{SYSTEM_HOME}/mydata5.json','w').write(f)


28704

### Define a small Plot.ly Dash web app using classes from the module ```dgrid_components.py```

In [21]:
no_border={
    'border': '1px solid #000',
    'grid-gap': '10px 10px',
}
df_save_list = []
logger = dgc.init_root_logger('logfile.log','INFO')

def dash_app_1(df_to_show,grid_template_columns_list):
    # Step 01: create a table component
    dt1_comp = dgc.DashTableComponent(
        'dt1',df_to_show,None,title='Random Walk',
        editable_columns=['close'],logger = logger
    )
    
    # Step 02: define a callback method that get's called from the gr1_comp dgc.FigureComponent components.
    #    The callback will extract a Pandas DataFrame in json format, 
    #    convert it to back to a DataFrame, create a Ploty Figure object, and return
    #    the figure object to the caller (the internal callback that Dash executes when
    #    it renders gr1_comp).
    def create_figure_from_df(input_list):
        if input_list is None or len(input_list)<1 or input_list[0] is None:
            return [None]
        dict_df = input_list[0]
        df = dgc.make_df(dict_df)
        df.close = df.close.astype(str).astype(float)
        p = dgc.PlotlyCandles(df,number_of_ticks_display=20,title='Random Walk').get_figure()
        return p
    
    # Step 03: Create a FigureComponent, which displays any Figure created with Plotly to be displayable 
    #           on your webpage.  Pass in the method create_figure_from_df method above, so that 
    #           this component can dynamically get data from the DashTableComponent and use it as 
    #           the input to a Plotly chart Figure that gets created 
    gr1_comp = dgc.FigureComponent('f1',dt1_comp,create_figure_from_df,logger=logger)

    # Step 04: Create a basic Dash app
    comp_list = [dt1_comp,gr1_comp]
    app = dgc.make_app(comp_list,grid_template_columns_list=grid_template_columns_list)
    return app


## Show several ways to launch a Dash App with different appearances
After each launch, hit the stop button to stop the cell from running ,and then run the next cell

### Launch 01  the web app as a page with 2 rows, one for the table and the next for the graph
##### To see the web app, open a browser to localhost:8500

In [22]:
days_to_show = 50
df_to_show = df_pseudo.iloc[-days_to_show:]  
# The grid_template_columns_list defines how the components appear on the page.  They use standard css values
#   that you use in a css grid
grid_template_columns_list = ['1fr','1fr']
app = dash_app_1(df_to_show,grid_template_columns_list)
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-10 17:05:10,653 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-10 17:05:13,739 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:05:13] "GET / HTTP/1.1" 200 -
2020-01-10 17:05:13,871 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:05:13] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:05:13,874 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:05:13] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:05:13,887 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:05:13] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:05:13,892 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:05:13] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1574289294 HTTP/1.1" 200 -
2020-01-10 17:05:13,888 - werkzeug - INFO - 127.0.0.1 

### Launch 02  the web app as a page with 1 row and 2 columns
##### To see the web app, open a browser to localhost:8500

In [23]:
# the grid_template_columns_list now has one element, that defines 2 equal size columns
grid_template_columns_list = ['1fr 1fr']
app = dash_app_1(df_to_show,grid_template_columns_list)
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-10 17:12:09,055 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-10 17:12:11,354 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:12:11] "GET / HTTP/1.1" 200 -
2020-01-10 17:12:11,551 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:12:11] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:12:11,551 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:12:11] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:12:11,552 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:12:11] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:12:11,569 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:12:11] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:12:11,581 - werkzeug - INFO - 

### Launch 02  the web app as a page with 1 row and 2 columns, but where the graph column is twice the size of the data column
##### To see the web app, open a browser to localhost:8500

In [24]:
# the grid_template_columns_list now has one element, that defines 2 columns, 
#    where the second is twice as big as the first
grid_template_columns_list = ['1fr 2fr']
app = dash_app_1(df_to_show,grid_template_columns_list)
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-10 17:13:37,395 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-10 17:13:40,162 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:13:40] "GET / HTTP/1.1" 200 -
2020-01-10 17:13:40,289 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:13:40] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:13:40,291 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:13:40] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:13:40,307 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:13:40] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:13:40,317 - werkzeug - INFO - 127.0.0.1 - - [10/Jan/2020 17:13:40] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-10 17:13:40,354 - werkzeug - INFO - 